In [ ]:
"""Not used in paper. Original legacy notebook"""
pip install torcheval

In [ ]:
pip install -U fvcore

In [ ]:
"""Import required libraries"""
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import math
from scipy.special import erfinv
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import random
from fvcore.nn import FlopCountAnalysis

In [ ]:
"""Set the training device to GPU"""
if torch.backends.mps.is_available():
  mps_device = torch.device("mps")
else:
  mps_device = torch.device("cpu")

In [ ]:
"""
Generate the Filter Attention Matrix referenced in Figure 1 of the Paper from a given filter, with a configurable stride, padding and default value.
"""

sd = 1
filter = torch.normal(1, 0, size = (3,3))
img_h = 8
img_w = 8
filt_h = 3
filt_w = 3
def gen_filter_attention(img_h, img_w, stride = 1, padding = 1, filter = filter, add = 0.0):
    filter_attention_matrix = torch.zeros((img_h*img_w,img_h*img_w))+add
    rolls = torch.linspace(0, img_w*img_h-1, img_w*img_h, dtype = torch.int64).repeat(img_h*img_w,1)
    offset_x = 1-padding
    offset_y = 1-padding
    calc_height = int((img_h+padding*2 - filter.shape[0]+1)/stride)
    calc_width = int((img_w+padding*2 - filter.shape[1]+1)/stride)
    if(calc_width > 0 and calc_height > 0):
        print(calc_height)
        print(calc_width)
        for i in range(calc_height):
            for j in range(calc_width):
                if offset_y == 0:
                    if offset_x == 0:
                        filter_attention_matrix[j+i*calc_width][0:2] = filter[1][1:3]
                        filter_attention_matrix[j+i*calc_width][img_w:img_w+2] = filter[2][1:3]
                    elif offset_x == img_w-1:
                        filter_attention_matrix[j+i*calc_width][offset_x-1:offset_x+1] = filter[1][0:2]
                        filter_attention_matrix[j+i*calc_width][img_w+offset_x-1:img_w+offset_x+1] = filter[2][0:2]
                    else:
                        filter_attention_matrix[j+i*calc_width][offset_x-1:offset_x+2] = filter[1][0:3]
                        filter_attention_matrix[j+i*calc_width][img_w+offset_x-1:img_w+offset_x+2] = filter[1][0:3]
                elif offset_y == img_w-1:
                    if offset_x == 0:
                        filter_attention_matrix[j+i*calc_width][(offset_y-1)*img_w:(offset_y-1)*img_w+2] = filter[0][1:3]
                        filter_attention_matrix[j+i*calc_width][(offset_y)*img_w:(offset_y)*img_w+2] = filter[1][1:3]
                    elif offset_x == img_w-1:
                        filter_attention_matrix[j+i*calc_width][(offset_y-1)*img_w+offset_x-1:(offset_y-1)*img_w+offset_x+1] = filter[0][0:2]
                        filter_attention_matrix[j+i*calc_width][(offset_y)*img_w+offset_x-1:(offset_y)*img_w+offset_x+1] = filter[1][0:2]
                    else:
                        filter_attention_matrix[j+i*calc_width][(offset_y-1)*img_w+offset_x-1:(offset_y-1)*img_w+offset_x+2] = filter[0][0:3]
                        filter_attention_matrix[j+i*calc_width][(offset_y)*img_w+offset_x-1:(offset_y)*img_w+offset_x+2] = filter[1][0:3]
                else:
                    if offset_x == 0:
                        filter_attention_matrix[j+i*calc_width][(offset_y-1)*img_w:(offset_y-1)*img_w+2] = filter[0][1:3]
                        filter_attention_matrix[j+i*calc_width][offset_y*img_w:offset_y*img_w+2] = filter[1][1:3]
                        filter_attention_matrix[j+i*calc_width][(offset_y+1)*img_w:(offset_y+1)*img_w+2] = filter[2][1:3]
                    elif offset_x == img_w-1:
                        filter_attention_matrix[j+i*calc_width][(offset_y-1)*img_w+offset_x-1:(offset_y-1)*img_w+offset_x+1] = filter[0][0:2]
                        filter_attention_matrix[j+i*calc_width][(offset_y)*img_w+offset_x-1:(offset_y)*img_w+offset_x+1] = filter[1][0:2]
                        filter_attention_matrix[j+i*calc_width][(offset_y+1)*img_w+offset_x-1:(offset_y+1)*img_w+offset_x+1] = filter[2][0:2]
                    else:
                        filter_attention_matrix[j+i*calc_width][(offset_y-1)*img_w+offset_x-1:(offset_y-1)*img_w+offset_x+2] = filter[0][0:3]
                        filter_attention_matrix[j+i*calc_width][(offset_y)*img_w+offset_x-1:(offset_y)*img_w+offset_x+2] = filter[1][0:3]
                        filter_attention_matrix[j+i*calc_width][(offset_y+1)*img_w+offset_x-1:(offset_y+1)*img_w+offset_x+2] = filter[2][0:3]
                filter_attention_matrix[j+i*calc_width] = torch.roll(filter_attention_matrix[j+i*calc_width],-(offset_x+offset_y*img_h))
                rolls[j+i*calc_width] = torch.roll(rolls[j+i*calc_width],(offset_x+offset_y*img_h))
                offset_x+=stride
            offset_x=0
            offset_y+=stride
    return filter_attention_matrix, rolls
filter_attention_matrix, rolls = gen_filter_attention(img_h = 8, img_w = 8, filter=filter)

In [ ]:
"""
Generate a matrix that has stronger activatoins for nearby pixels in the image, and weaker activations for pixels farther away to induce locality bias. Distance is calcuated by L2 Norm between pixels of image. Not used in the paper.
"""
def gen_radial(img_h, img_w, scale = 0.5, stride = 1):
    rolls = torch.linspace(0, img_w*img_h-1, img_w*img_h, dtype = torch.int64).repeat(img_h*img_w,1)
    radial_attention_matrix = torch.zeros((img_h*img_w,img_h*img_w))
    calc_w = int(img_w/stride)
    calc_h = int(img_h/stride)
    x_pos = 0
    y_pos = 0
    for curr_i in range(calc_h):
        for curr_j in range(calc_w):
            curr_matrix = torch.zeros((img_h, img_w))
            for i in range(img_h):
                for j in range(img_w):
                    euc_weight = 1/np.exp(scale*(((y_pos-i)**2+(x_pos-j)**2)**(0.5)))
                    curr_matrix[i][j] = euc_weight
            radial_attention_matrix[curr_i*calc_h+curr_j] = torch.roll(curr_matrix.reshape(-1),-(y_pos*img_w+x_pos))
            rolls[curr_i*calc_h+curr_j] = torch.roll(rolls[curr_i*calc_h+curr_j],(y_pos*img_w+x_pos))
            x_pos+=stride
        x_pos = 0
        y_pos+=stride
    return radial_attention_matrix, rolls
radial_attention_matrix = gen_radial(img_h = 8, img_w = 8,)

In [ ]:
"Generate a matrix of all ones. Induces no initial inductive bias on the Learned Mask Self Attention layer. Not used in the paper"
def gen_ones(img_h, img_w, stride = 1):
    rolls = torch.linspace(0, img_w*img_h-1, img_w*img_h, dtype = torch.int64).repeat(img_h*img_w,1)
    ones_attention_matrix = torch.normal(1,0.1,(img_h*img_w,img_h*img_w))
    calc_w = int(img_w/stride)
    calc_h = int(img_h/stride)
    x_pos = 0
    y_pos = 0
    for curr_i in range(calc_h):
        for curr_j in range(calc_w):
            rolls[curr_i*calc_h+curr_j] = torch.roll(rolls[curr_i*calc_h+curr_j],(y_pos*img_w+x_pos))
            x_pos+=stride
        x_pos = 0
        y_pos+=stride
    return ones_attention_matrix, rolls
ones_attention_matrix = gen_ones(img_h = 8, img_w = 8)

In [ ]:
"""
Generate a matrix that has stronger activatoins for nearby pixels in the image, and weaker activations for pixels farther away to induce locality bias. Distance is calcuated by L1 Norm between pixels of image. Not used in the paper.
"""
def gen_l1(img_h, img_w, scale = 0.5, stride = 1):
    rolls = torch.linspace(0, img_w*img_h-1, img_w*img_h, dtype = torch.int64).repeat(img_h*img_w,1)
    radial_attention_matrix = torch.zeros((img_h*img_w,img_h*img_w))
    l1_attention_matrix = torch.zeros((img_h*img_w, img_h*img_w))
    calc_w = int(img_w/stride)
    calc_h = int(img_h/stride)
    x_pos = 0
    y_pos = 0
    for curr_i in range(calc_h):
        for curr_j in range(calc_w):
            curr_matrix = torch.zeros((img_h, img_w))
            for i in range(img_h):
                for j in range(img_w):
                    euc_weight = 1/np.exp(scale*(abs(y_pos-i)+abs(x_pos-j)))
                    curr_matrix[i][j]=euc_weight
            l1_attention_matrix[curr_i*calc_h+curr_j] = torch.roll(curr_matrix.reshape(-1),-(y_pos*img_w+x_pos))
            rolls[curr_i*calc_h+curr_j] = torch.roll(rolls[curr_i*calc_h+curr_j],(y_pos*img_w+x_pos))
            x_pos+=stride
        x_pos = 0
        y_pos+=stride
    return l1_attention_matrix, rolls
l1_attention_matrix = gen_l1(img_h = 8, img_w = 8)

In [ ]:
import torch.nn as nn
import torch
"""Module that allows for the training two different masks, and using rank approximation to mimic the filter attention map."""
class MaskTrainer(nn.Module):
    def __init__(self, seq_len = 1024, repeat_masks = True, num_masks = 16, mask_fidelity = 9, groups = 5):
        super().__init__()
        if(repeat_masks):
            self.mask1 = nn.Parameter(torch.normal(0.0,0.001,(mask_fidelity, seq_len)), requires_grad = True)
            self.mask2 = nn.Parameter(torch.normal(0.0,0.001,(mask_fidelity, seq_len)), requires_grad = True)
        else:
            self.mask1 = nn.Parameter(torch.normal(0.0,0.001,(groups, num_masks, mask_fidelity, seq_len)), requires_grad = True)
            self.mask2 = nn.Parameter(torch.normal(0.0,0.001,(groups, num_masks, mask_fidelity, seq_len)), requires_grad = True)
    def forward(self, inputs):
        att = torch.matmul(self.mask1.transpose(-1,-2), self.mask2)
        return att
model = MaskTrainer()

In [ ]:
"""Module that allows for double rank approximation. Using two masks to approximate each mask in Rank Approximation. Those two computed masks are then used to mimic filter attention map. Not used in paper."""
class DoubleMaskTrainer(nn.Module):
    def __init__(self, img_h = 32, img_w = 32, repeat_masks = True, num_masks = 16, mask_fidelity1 = 4, mask_fidelity2 = 9):
        super().__init__()
        self.mask_fidelity1 = mask_fidelity1
        self.mask_fidelity2 = mask_fidelity2
        if(repeat_masks):
            self.mask1 = nn.Parameter(torch.normal(0.0,0.1,(mask_fidelity2,mask_fidelity1, img_h)), requires_grad = True)
            self.mask2 = nn.Parameter(torch.normal(0.0,0.1,(mask_fidelity2,mask_fidelity1, img_w)), requires_grad = True)
            self.mask3 = nn.Parameter(torch.normal(0.0,0.1,(mask_fidelity2,mask_fidelity1, img_h)), requires_grad = True)
            self.mask4 = nn.Parameter(torch.normal(0.0,0.1,(mask_fidelity2,mask_fidelity1, img_w)), requires_grad = True)
        else:
            self.mask1 = nn.Parameter(torch.normal(0.0,0.1,(num_masks, mask_fidelity2,mask_fidelity1, img_h)), requires_grad = True)
            self.mask2 = nn.Parameter(torch.normal(0.0,0.1,(num_masks,mask_fidelity2,mask_fidelity1, img_w)), requires_grad = True)
            self.mask3 = nn.Parameter(torch.normal(0.0,0.1,(num_masks,mask_fidelity2,mask_fidelity1, img_h)), requires_grad = True)
            self.mask4 = nn.Parameter(torch.normal(0.0,0.1,(num_masks,mask_fidelity2,mask_fidelity1, img_w)), requires_grad = True)
    def forward(self, inputs):
        att = torch.matmul(torch.matmul(self.mask1.transpose(-1,-2), self.mask2).reshape(self.mask_fidelity2,-1).transpose(-1,-2), torch.matmul(self.mask3.transpose(-1,-2), self.mask4).reshape(self.mask_fidelity2, -1))
        return att
model = MaskTrainer()

In [ ]:
"""
Function that executes training procedure for the Learnable Masks using MaskTrainer, generating the trained masks that mimic the filter attention map, and are used for rank approximation.
"""
def gen_rolls_and_masks(img_sizes, patch_size = 2, filt_type = 'filter', train_type = "single", should_train = False, mask_fidelity = 9):
    img_sizes = [(int(img_h/patch_size), int(img_w/patch_size)) for img_h, img_w in img_sizes]
    rolls = []
    masks = []
    for j, (img_h, img_w) in enumerate(img_sizes):
        filter = torch.normal(1, 0, size = (3,3))
        if filt_type == 'filter':
            print(f"img_h = {img_h} + img_w = {img_w}")
            attention_matrix, roll_data = (data.to(mps_device) for data in gen_filter_attention(filter=filter, img_h = img_h, img_w = img_w, stride = 1))
            rolls.append(roll_data)
        elif filt_type == 'radial':
            print(f"img_h = {img_h} + img_w = {img_w}")
            attention_matrix, roll_data = (data.to(mps_device) for data in gen_radial(img_h = img_h, img_w = img_w, stride = 1))
            rolls.append(roll_data)
        elif filt_type == 'ones':
            print(f"img_h = {img_h} + img_w = {img_w}")
            attention_matrix, roll_data = (data.to(mps_device) for data in gen_ones(img_h = img_h, img_w = img_w, stride = 1))
            rolls.append(roll_data)
        else:
            print(f"img_h = {img_h} + img_w = {img_w}")
            attention_matrix, roll_data = (data.to(mps_device) for data in gen_l1(img_h = img_h, img_w = img_w, stride = 1))
            rolls.append(roll_data)
        plt.matshow(attention_matrix.cpu())
        plt.colorbar()
        plt.show()
        model = MaskTrainer(seq_len = img_h*img_w, mask_fidelity = mask_fidelity).to(mps_device) if train_type == "single" else DoubleMaskTrainer(img_h = img_h, img_w = img_w).to(mps_device)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
        loss_fn = nn.MSELoss()
        loop_num = 10000
        if(should_train):
            for i in range(loop_num):
                optimizer.zero_grad()
                outputs = model(1)
                loss = loss_fn(outputs, attention_matrix)
                if i % 2000 == 0:
                    print(loss)
                loss.backward()
                optimizer.step()
        if train_type == "single":
            mask1 =  model.mask1
            mask2 = model.mask2
            masks.append((mask1, mask2))
        else:
            mask1 =  model.mask1
            mask2 = model.mask2
            mask3 = model.mask3
            mask4 = model.mask4
            masks.append((mask1, mask2, mask3, mask4))
    return (rolls, masks)

rolls, masks = (torch.zeros((3,3)),torch.zeros((3,3)))
rolls, masks = gen_rolls_and_masks(img_sizes = [(32,32), (16,16), (8,8)], filt_type = 'filter',  mask_fidelity=9)

In [ ]:
"""
Function that executes training procedure for the Learnable Masks using MaskTrainer. All the masks are slightly different, since different architecutres might need slight differences in the mask matrix to break symetries during training. Not used in the paper.
"""
import copy
img_sizes = []
for i in range(5): img_sizes.append((32, 32))
for i in range(3): img_sizes.append((16, 16))
for i in range(1): img_sizes.append((8,8))
strides = []
for i in range(3): strides.append(1)
for i in range(2): strides.append(2)
for i in range(1): strides.append(1)
for i in range(2): strides.append(2)
for i in range(1): strides.append(1)
block1_masks = 32
block2_masks = 64
block3_masks = 128
num_masks = []
for i in range(3): num_masks.append(block1_masks)
for i in range(3): num_masks.append(block2_masks)
for i in range(3): num_masks.append(block3_masks)
def gen_rolls_and_masks_without_repeat(img_sizes = img_sizes, strides = strides, patch_size = 4, num_masks = num_masks, mask_fidelity = 9, groups = 3):
    img_sizes = [(int(img_h/math.sqrt(patch_size)), int(img_w/math.sqrt(patch_size))) for img_h, img_w in img_sizes]
    filters = [torch.normal(1,0.1,(groups, num_masks, 3,3)) for i, num_masks in enumerate(num_masks)]
    rolls = []
    masks = []
    for j, (img_h, img_w) in enumerate(img_sizes):
        print(f"img_h = {img_h} + img_w = {img_w}")
        attention_matrix = torch.stack([torch.stack([gen_filter_attention(filter=filter, img_h = img_h, img_w = img_w, stride = strides[j])[0].to(mps_device) for filter in filter_row]) for filter_row in filters[j]])
        print(f"shape:{attention_matrix.shape}")
        roll_data = gen_filter_attention(filter=filters[j][0,0], img_h = img_h, img_w = img_w, stride = strides[j])[1].to(mps_device)
        rolls.append(roll_data)
        plt.matshow(attention_matrix[0,0].detach().cpu())
        plt.matshow(roll_data.detach().cpu())
        plt.colorbar()
        plt.show()
        model = MaskTrainer(seq_len = img_h*img_w, repeat_masks=False, num_masks = num_masks[j], mask_fidelity = mask_fidelity, groups = groups).to(mps_device)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)
        loss_fn = nn.MSELoss()
        loop_num = 10000
        for i in range(loop_num):
            optimizer.zero_grad()
            outputs = model(1)
            loss = loss_fn(outputs, attention_matrix)
            if i % 2000 == 0:
                print(loss)
            loss.backward()
            optimizer.step()
        mask1 =  model.mask1
        mask2 = model.mask2
        masks.append((mask1, mask2))
        seq_len = img_w*img_h
    return (rolls, masks)
rolls, masks = gen_rolls_and_masks_without_repeat()

<h2><b>CIFAR 10 Performance</b></h2>

In [ ]:
"""Loads the CIFAR-10 Dataset, and splits it into two sections. This allows for reducing dataset size to train models on smaller dataset sizes and analyze model scalability."""
CIFAR10Orig = torchvision.datasets.CIFAR10("Datasets/CIFAR-10",train=True, download=True)
seed = 1
CIFAR10, CIFAR_Val = torch.utils.data.random_split(
        CIFAR10Orig, [40000, 10000], generator=torch.Generator().manual_seed(seed))

print(len(CIFAR10))
CIFAR_Train = CIFAR10

CIFAR_Test = torchvision.datasets.CIFAR10("Datasets/CIFAR-10", train = False, download=True)
print(len(CIFAR_Val))

In [ ]:
"""Pytorch Dataset class that allows for easy and efficient dataloading of the CIFAR-10 dataset. Has the ability to downsample images for training without patching."""
class CIFAR_Conv(Dataset):
    def __init__(self, Dataset = CIFAR_Train, train = True):
        self.CIFAR = Dataset
        if(train):
          self.tensor_transform = transforms.Compose([
              transforms.Resize((32,32)),
              transforms.ToTensor(),
              transforms.Normalize(0,1)
          ])
        else:
          self.tensor_transform = transforms.Compose([
              transforms.Resize((32,32)),
              transforms.ToTensor(),
              transforms.Normalize(0,1)
          ])
    def __getitem__(self, index):
        image = self.CIFAR.__getitem__(index)
        return (self.tensor_transform(image[0]),image[1])
    def __len__(self):
        return len(self.CIFAR)

In [ ]:
"""Initialize all the datasets and dataloaders"""
CIFAR_Convolve = CIFAR_Conv()
CIFAR_Conv_Val = CIFAR_Conv(Dataset = CIFAR_Val, train = False)
CIFAR_Convloader = DataLoader(CIFAR_Convolve, batch_size = 180, shuffle = True, pin_memory = False)
CIFAR_Valconvloader = DataLoader(CIFAR_Conv_Val, batch_size = 180, shuffle = True)

In [ ]:
"""Configure pytorch to access unused VRAM already allocated by another program. This allows for the model to use every last bit of VRAM available on the GPU."""
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
"""
Conduct patching described by paper. Spreads the pixels in each non-overlapping patch along the channel dimension.
"""
class Patchify(nn.Module):
    def __init__(self, patch_size = 2):
        super().__init__()
        self.patch_size = patch_size
        self.unfold = nn.Unfold(patch_size, stride=patch_size)
    def forward(self, input):
        return self.unfold(input).transpose(-1,-2)

In [ ]:
"""
Computes Batch Normalization for the ConvFormer, Vision Transformer, and the Transformer.
"""
class BatchNormTranspose(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.BNorm = nn.BatchNorm1d(channels)
    def forward(self, input):
        output = self.BNorm(input.transpose(-1,-2))
        return output.transpose(-1,-2)

In [ ]:
"""
Appends fixed sine, and cosine positional encodings to Vision Transformer and ConvFormer.
"""
class PositionalEncodingAppend(nn.Module):
    def __init__(self, total_size, device=mps_device):
        super().__init__()
        self.total_size = total_size
        self.device = device
    def forward(self, input):
        positions = torch.arange(input.shape[-2],dtype=torch.float32).view(-1, 1).to(self.device)
        freq = 1/torch.exp(torch.linspace(0,8,int((self.total_size-input.shape[-1])/2), dtype=torch.float32)).view(1,-1).to(self.device)
        sin_encodings = torch.sin(torch.matmul(positions, freq)).repeat(input.shape[0],1,1)
        cos_encodings = torch.cos(torch.matmul(positions, freq)).repeat(input.shape[0],1,1)
        return torch.cat((input, sin_encodings, cos_encodings), -1)


In [ ]:
"""This is the implementation of the Learned Masked Self Attention Layer described in the paper. It is implemented according to the algorithm detailed in the paper."""
import math
import torch.autograd.profiler as profiler
import gc
class MultiScaledConvHead(nn.Module):
    def __init__(self, num_heads, d_model, mask1, mask2, num_masks, roll_matrix, reduction = False, patch_size = 2, repeat_masks = True):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.num_masks = num_masks
        self.Linears = nn.ModuleList(nn.Linear(d_model, d_model) for __ in range(3))
        if(repeat_masks):
            self.mask1 = nn.Parameter(mask1.clone().repeat(num_masks, 1, 1), requires_grad = True)
            self.mask2 = nn.Parameter(mask2.clone().repeat(num_masks, 1, 1), requires_grad = True)
        else:
            self.mask1 = nn.Parameter(mask1.clone(), requires_grad = True)
            self.mask2 = nn.Parameter(mask2.clone(), requires_grad = True)
        self.register_buffer(name='roll_back', tensor=roll_matrix.clone())
        self.roll_back = self.roll_back
        self.reduction = reduction
        self.patch_size = patch_size
        self.patchify = Patchify(patch_size=2)
        self.epsilon = 0.00000001
        self.final_linear = nn.Linear(d_model, d_model)
    def forward(self, input):
        keys, queries, values = (
            linear(inp).reshape(input.shape[0], -1, self.num_heads, int(self.d_model/self.num_heads)).transpose(1,2)
            for linear, inp in zip(self.Linears, (input, input, input))
        )
        learnable_mask = torch.matmul(self.mask1.transpose(-1,-2), self.mask2)
        # print(learnable_mask.shape)
        att = torch.matmul(queries,keys.transpose(-1,-2))/math.sqrt(self.d_model/self.num_heads)
        att = att.reshape(input.shape[0], int(self.num_heads/self.num_masks),self.num_masks, att.shape[-2],att.shape[-1])
        del keys
        del queries
        att = att*torch.gather(learnable_mask,-1,self.roll_back.expand_as(learnable_mask)).expand_as(att)
        # print(att.shape)
        del learnable_mask
        att = att.reshape(input.shape[0], self.num_heads, att.shape[-2], att.shape[-1])
        #Final attention normed to prevent exploding and vanishing gradients.
        att = att/(torch.norm(att, dim = (-1,-2))[..., None,None]+self.epsilon)
        outputs = torch.matmul(att,values).transpose(1,2).reshape(input.shape[0], -1, self.d_model)
        outputs = self.final_linear(outputs)
        if self.reduction:
            outputs = outputs.reshape(outputs.shape[0], int(math.sqrt(outputs.shape[-2])),int(math.sqrt(outputs.shape[-2])), outputs.shape[-1])
            outputs = outputs.permute(0,3,1,2)
            outputs = self.patchify(outputs)
        del att
        del values
        return outputs
gc.collect()


In [ ]:
"""Test LMSA Layer"""
head = MultiScaledConvHead(num_heads = 16, d_model = 32, reduction = False, mask1=masks[0][0], mask2=masks[0][0], num_masks = 16, roll_matrix=rolls[0]).to(mps_device)
input = torch.zeros(10, 256, 32).to(mps_device)
print(input)
output = head(input)
print(output.shape)
del head 
del input
del output
gc.collect()

In [ ]:
import math
import torch.autograd.profiler as profiler
import gc
"""Allow more masks per head in LMSA to copmletely represent convolution. Not covered in paper."""
class MultiScaledConvFilterHead(nn.Module):
    def __init__(self, scale = 2, num_heads = 16, d_model = 48, seq_len = 256, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0], reduction = 64, groups = 1):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.scale = scale
        self.Linears = nn.ModuleList(nn.Linear(d_model, scale*d_model) for __ in range(3))
        self.mask1 = nn.Parameter(mask1.clone(), requires_grad = True)
        self.mask2 = nn.Parameter(mask2.clone(), requires_grad = True)
        self.register_buffer(name='roll_back', tensor=roll_matrix.clone())
        self.roll_back = self.roll_back[...,:reduction,:]
        self.reduction = reduction
        self.groups = groups
        self.epsilon = 0.00000001
    def forward(self, input):
        with profiler.record_function("KEY_QUERY_VALUE_PROJECTION"):
            keys, queries, values = (
                linear(inp).reshape(input.shape[0], -1, self.num_heads, int(self.d_model*self.scale/self.num_heads)).transpose(1,2)
                for linear, inp in zip(self.Linears, (input, input, input))
            )

        with profiler.record_function("LEARNABLE_MASK_GEN"):
            learnable_mask = torch.matmul(self.mask1.transpose(-1,-2), self.mask2)[...,:self.reduction,:]
        with profiler.record_function("ATTENTION_MATMUL"):
            att = torch.matmul(queries,keys.transpose(-1,-2))[...,:self.reduction,:]/math.sqrt(self.d_model*self.scale/self.num_heads)

        with profiler.record_function("APPLY_LEARNABLE_MASK"):

            att = att.reshape(input.shape[0], 1, 1, self.num_heads, att.shape[-2],att.shape[-1]).expand(-1,self.groups,int(self.d_model*self.scale/self.num_heads), -1,-1,-1)

            learnable_mask = learnable_mask.reshape(self.groups, self.num_heads, -1, learnable_mask.shape[-2], learnable_mask.shape[-1]).transpose(1,2)
            del keys
            del queries
            att = att*torch.gather(learnable_mask,-1,self.roll_back.expand_as(learnable_mask)).expand_as(att)
            att = torch.sum(att.transpose(2,3).reshape(input.shape[0], self.groups, self.d_model*self.scale, att.shape[-2], att.shape[-1]),dim=1)

            del learnable_mask


            att = att/(torch.norm(att, dim = (-1,-2))[..., None,None]+self.epsilon)

        values = values.transpose(1,2).reshape(input.shape[0], -1, self.d_model*self.scale).transpose(-1,-2).reshape(input.shape[0], self.d_model*self.scale, -1, 1)
        outputs = torch.matmul(att, values)
        outputs = outputs
        outputs = outputs.reshape(input.shape[0], self.d_model*self.scale, -1).transpose(-1,-2)
        del att
        del values

        return outputs
gc.collect()

In [ ]:
"""Test Multi-Filter LMSA Layer"""
head = MultiScaledConvFilterHead().to(mps_device)
input = torch.zeros(1, 256, 48).to(mps_device)
for i in range(48):
    input[:,:,i] = torch.full((1,256),i).to(mps_device)
print(input)
outputs = head(input)
print(outputs.shape)
print(outputs)

In [ ]:
"""
LMSA Layer with double rank approximation, using DoubleMaskTrainer to retrieve weights. Not used in Paper.
"""
class DoubleMultiScaledConvHead(nn.Module):
    def __init__(self, scale = 45, num_heads = 9, d_model = 1, seq_len = 784, mask1 = masks[0][0], mask2 = masks[0][1], mask3 = masks[0][2], mask4 = masks[0][3], num_masks = 1, roll_matrix = rolls[0], reduction = 784, repeat_masks = False):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.scale = scale
        self.num_masks = num_masks
        self.Linears = nn.ModuleList(nn.Linear(d_model, scale*d_model) for __ in range(3))
        if(repeat_masks):
            self.mask1 = nn.Parameter(mask1.clone().repeat(num_masks, 1, 1), requires_grad = True)
            self.mask2 = nn.Parameter(mask2.clone().repeat(num_masks, 1, 1), requires_grad = True)
            self.mask3 = nn.Parameter(mask3.clone().repeat(num_masks, 1, 1), requires_grad = True)
            self.mask4 = nn.Parameter(mask4.clone().repeat(num_masks, 1, 1), requires_grad = True)
        else:
            self.mask1 = nn.Parameter(mask1.clone(), requires_grad = True)
            self.mask2 = nn.Parameter(mask2.clone(), requires_grad = True)
            self.mask3 = nn.Parameter(mask1.clone(), requires_grad = True)
            self.mask4 = nn.Parameter(mask2.clone(), requires_grad = True)
        self.register_buffer(name='roll_back', tensor=roll_matrix.clone())
        self.roll_back = self.roll_back[...,:reduction,:]
        self.mult1 = nn.Parameter(torch.full((1,1),-2.197), requires_grad = True)
        self.mult2 = nn.Parameter(torch.full((1,1),2.197), requires_grad = True)
        self.reduction = reduction
    def forward(self, input):
        with profiler.record_function("KEY_QUERY_VALUE_PROJECTION"):
            keys, queries, values = (
                linear(inp).reshape(input.shape[0], self.num_heads, -1, int(self.d_model*self.scale/self.num_heads))
                for linear, inp in zip(self.Linears, (input, input, input))
            )
        with profiler.record_function("LEARNABLE_MASK_GEN"):
            learnable_mask = torch.matmul(torch.matmul(self.mask1.transpose(-1,-2), self.mask2).reshape(self.mask_fidelity2,-1).transpose(-1,-2), torch.matmul(self.mask3.transpose(-1,-2), self.mask4).reshape(self.mask_fidelity2, -1))[...,:self.reduction,:]
        with profiler.record_function("ATTENTION_MATMUL"):
            att = torch.matmul(queries,keys.transpose(-1,-2))[...,:self.reduction,:]/math.sqrt(self.d_model*self.scale/self.num_heads)

        with profiler.record_function("APPLY_LEARNABLE_MASK"):
            att = att.reshape(input.shape[0], int(self.num_heads/self.num_masks), self.num_masks, att.shape[-2],att.shape[-1])
            del keys
            del queries
            att = nn.functional.sigmoid(self.mult1)*att+nn.functional.sigmoid(self.mult2)*torch.gather(learnable_mask,2,self.roll_back.expand_as(learnable_mask)).expand_as(att)
            del learnable_mask
            att = att.reshape(input.shape[0], -1, att.shape[-2], att.shape[-1])
        outputs = torch.matmul(att,values).view(input.shape[0], self.reduction, -1)
        del att
        del values
        return outputs
gc.collect()


In [ ]:
"""
Multi-Head Attention Layer detailed in Attention is All You Need (Vaswani, 2017) paper. Implemented to compare Transformers with the same ConvFormer architecture and Vision Transformers to ConvFormers and ResNets.
"""
import math
import torch.autograd.profiler as profiler
import gc
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, d_model):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.Linears = nn.ModuleList(nn.Linear(d_model, d_model) for __ in range(3))
        self.final_linear = nn.Linear(d_model, d_model)
    def forward(self, input):
        with profiler.record_function("KEY_QUERY_VALUE_PROJECTION"):
            input = input.reshape(input.shape[0], self.d_model, -1).transpose(-1,-2)  # Transpose to (batch_size, seq_len, d_model)
            keys, queries, values = (
                linear(inp).reshape(input.shape[0], -1, self.num_heads, int(self.d_model/self.num_heads)).transpose(1,2)
                for linear, inp in zip(self.Linears, (input, input, input))
            )
        with profiler.record_function("ATTENTION_MATMUL"):
            att = torch.matmul(queries,keys.transpose(-1,-2))/math.sqrt(self.d_model/self.num_heads)
            att = torch.nn.functional.softmax(att, dim=-1)
        outputs = torch.matmul(att,values).transpose(1,2).reshape(input.shape[0], -1, self.d_model)
        outputs = self.final_linear(outputs)
        return outputs

model = MultiHeadAttention(num_heads=16, d_model=32)
gc.collect()

In [ ]:
"""
Multi-Head Attention Layer detailed in Attention is All You Need (Vaswani, 2017) paper. Implemented to compare Transformers with the same ConvFormer architecture and Vision Transformers to ConvFormers and ResNets.
"""
import math
import torch.autograd.profiler as profiler
import gc
class MultiHeadAttentionWReduction(nn.Module):
    def __init__(self, num_heads, d_model, patch_size = 1, reduction = False):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.patch_size = patch_size
        self.Linears = nn.ModuleList(nn.Linear(d_model, d_model) for __ in range(3))
        self.final_linear = nn.Linear(self.d_model, self.d_model)
        self.reduction = reduction
        self.patchify = Patchify(patch_size=patch_size)
    def forward(self, input):
        with profiler.record_function("KEY_QUERY_VALUE_PROJECTION"):
            keys, queries, values = (
                linear(inp).reshape(input.shape[0], -1, self.num_heads, int(self.d_model/self.num_heads)).transpose(1,2)
                for linear, inp in zip(self.Linears, (input, input, input))
            )
        with profiler.record_function("ATTENTION_MATMUL"):
            att = torch.matmul(queries,keys.transpose(-1,-2))/math.sqrt(self.d_model/self.num_heads)
            att = torch.nn.functional.softmax(att, dim=-1)
        outputs = torch.matmul(att,values).transpose(1,2).reshape(input.shape[0], -1, self.d_model)
        outputs = self.final_linear(outputs)
        if self.reduction:
            outputs = outputs.reshape(outputs.shape[0], int(math.sqrt(outputs.shape[-2])),int(math.sqrt(outputs.shape[-2])), outputs.shape[-1])
            outputs = outputs.permute(0,3,1,2)
            outputs = self.patchify(outputs)
        return outputs

model = MultiHeadAttention(num_heads=16, d_model=32)
gc.collect()

In [ ]:
"""
Test Multi-Head Attention Layer
"""
head = MultiHeadAttentionWReduction(num_heads=16, d_model=32, reduction=True, patch_size=2).to(mps_device)
input = torch.zeros(1, 256, 32).to(mps_device)
print(input)
outputs = head(input)
print(outputs.shape)
print(outputs)


In [ ]:
"""Implementation of one VisionTransformer block as described in the Vision Transformer paper (Dosovitsky, 2021)."""
class ViTBlock(nn.Module):
    def __init__(self, scale = 1, num_heads = 4, d_model = 1):
        super().__init__()
        self.BNorm1 = nn.LayerNorm(d_model)
        self.attention = MultiHeadAttention(num_heads = num_heads, d_model = d_model)
        self.BNorm2 = nn.LayerNorm(d_model)
        self.MLP = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.ReLU(),
            nn.Linear(4*d_model, d_model)
        )
    def forward(self, input):
        out0 = self.BNorm1(input)
        out1 = self.attention(out0)+out0
        out2 = self.BNorm2(out1)
        out3 = self.MLP(out2)+out2
        return out3


In [ ]:
"""Implements full Vision Transformer described in Vision Transformer paper."""
img_h = 32
img_w = 32
class VisionTransformer(nn.Module):
    def __init__(self, img_h, img_w):
        super().__init__()
        patch_size = 2
        d_model1 = 96
        self.conv_scale_up = nn.Sequential(
            nn.Conv2d(3,8,3,padding=1),
            Patchify(patch_size = patch_size),
            PositionalEncodingAppend(d_model1, device = mps_device)
        )
        block_list = [ViTBlock(num_heads = 16, d_model = d_model1) for i in range(8)]
        self.ViTBlocks = nn.Sequential(
            *block_list
        )
        self.fcn = nn.Sequential(
            nn.Linear(d_model1*int(img_h*img_w/(patch_size**2)), 10),
        )
    def forward(self, input):
        out0 = self.conv_scale_up(input)
        out1 = self.ViTBlocks(out0)
        out1 = out1.reshape(out0.shape[0], -1)
        out2 = self.fcn(out1)
        return out1
model = VisionTransformer(img_h = 32, img_w = 32).to(mps_device)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

In [ ]:
"""Test Vision Transformer"""
model = VisionTransformer(img_h = 32, img_w = 32).to(mps_device)
conv_input = torch.stack([CIFAR_Convolve.__getitem__(i)[0] for i in range(16)]).to(mps_device)
print(conv_input.shape)
conv_output = model(conv_input)
%timeit model(conv_input)

In [ ]:
"""Implements Transformer with the same model architecture as ResNets and ConvFormers to show that architectural differences are not responsible for the difference in performance between Transformers and ConvFormers."""
patch_size = 4
class TransformerResNet(nn.Module):
    def __init__(self, img_h, img_w):
        super().__init__()
        res_block1_heads = 16
        res_block2_heads = 32
        res_block3_heads = 64
        #Should be square number
        patch_size = 2
        d_model1 = 32
        d_model2 = d_model1*(patch_size**2)
        d_model3 = d_model2*(patch_size**2)
        dropout_p = 0.01
        self.conv_scale_up = nn.Sequential(
            nn.Conv2d(3,4,3,padding=1),
            Patchify(patch_size = patch_size),
            PositionalEncodingAppend(d_model1, device = mps_device)
        )
        self.res_block_11 = nn.Sequential(
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_12 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_13 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_14 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection1 = MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = True, patch_size = 2)
        self.res_block_21 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = True, patch_size = 2),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = False),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_22 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = False),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = False),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_23 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = False),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = False),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection2 = MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = True, patch_size = 2)
        self.res_block_31 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = True, patch_size = 2),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block3_heads, d_model = d_model3, reduction = False),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_32 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block3_heads, d_model = d_model3, reduction = False),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block3_heads, d_model = d_model3, reduction = False),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_33 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block3_heads, d_model = d_model3, reduction = False),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block3_heads, d_model = d_model3, reduction = False),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.fcn = nn.Sequential(
            nn.Linear(d_model3, 10)
        )
        self.LogSoftmax = nn.LogSoftmax(dim=1)
    def forward(self, input):
        out0 = self.conv_scale_up(input)
        out1 = self.res_block_11(out0)+out0
        out2 = self.res_block_12(out1)+out1
        out3 = self.res_block_13(out2)+out2
        out4 = self.res_block_14(out3)+out3
        out5 = self.res_block_21(out4)+self.shortcut_projection1(out4)
        out6 = self.res_block_22(out5)+out5
        out7 = self.res_block_23(out6)+out6
        out8 = self.res_block_31(out7)+self.shortcut_projection2(out7)
        out9 = self.res_block_32(out8)+out8
        out10 = self.res_block_33(out9)+out9
        out10 = torch.mean(out10, dim=-2)
        out11 = self.fcn(out10)
        out11 = out11 - torch.amax(out11, 1, keepdim=True)
        out11 = self.LogSoftmax(out11)
        return out11
model = TransformerResNet(img_h=32, img_w=32).to(mps_device)

In [ ]:
"""
Implements Reduced-Size Transformer with the same model architecture as ResNets and ConvFormers.
"""
patch_size = 2
class TransformerResNetSmall(nn.Module):
    def __init__(self, img_h, img_w, patch_size = 2):
        super().__init__()
        res_block1_heads = 16
        res_block2_heads = 32
        res_block3_heads = 64
        #Should be square number
        patch_size = 2
        d_model1 = 32
        d_model2 = d_model1*(patch_size**2)
        d_model3 = d_model2*(patch_size**2)
        dropout_p = 0.01
        self.conv_scale_up = nn.Sequential(
            nn.Conv2d(3,4,3,padding=1),
            Patchify(patch_size = patch_size),
            PositionalEncodingAppend(d_model1, device = mps_device)
        )
        self.res_block_11 = nn.Sequential(
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_12 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = False),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection1 = MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = True, patch_size=patch_size)
        self.res_block_21 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block1_heads, d_model = d_model1, reduction = True, patch_size=patch_size),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = False),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection2 = MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = True, patch_size=patch_size)
        self.res_block_31 = nn.Sequential(
            nn.ReLU(),
            MultiHeadAttentionWReduction(num_heads = res_block2_heads, d_model = d_model2, reduction = True, patch_size=patch_size),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiHeadAttentionWReduction(num_heads = res_block3_heads, d_model = d_model3, reduction = False),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.fcn = nn.Sequential(
            nn.Linear(d_model3, 10)
        )
        self.LogSoftmax = nn.LogSoftmax(dim=1)
    def forward(self, input):
        out0 = self.conv_scale_up(input)

        out1 = self.res_block_11(out0)+out0

        out2 = self.res_block_12(out1)+out1

        out3 = self.res_block_21(out2)+self.shortcut_projection1(out2)

        out4 = self.res_block_31(out3)+self.shortcut_projection2(out3)

        out5 = torch.mean(out4, dim=-2)

        out6 = self.fcn(out5)

        out6 = out6 - torch.amax(out6, 1, keepdim=True)
        out6 = self.LogSoftmax(out6)
        return out6
model = TransformerResNetSmall(img_h=32, img_w=32).to(mps_device)

In [ ]:
"""Implementation for ConvFormers, the main model presented in the paper."""
patch_size = 2
rolls, masks = gen_rolls_and_masks(img_sizes=[(32,32),(16,16), (8,8)], patch_size = patch_size, filt_type = "filter", mask_fidelity = 3, should_train=True)
class ConvFormerResNet(nn.Module):
    def __init__(self, img_h, img_w, patch_size, masks = masks, rolls = rolls):
        super().__init__()
        res_block1_heads = 16
        res_block2_heads = 32
        res_block3_heads = 64
        res_block1_masks = 16
        res_block2_masks = 32
        res_block3_masks = 64
        #Should be square number
        self.patch_size = patch_size
        self.reduce_patch_size = 2
        d_model1 = 32
        d_model2 = d_model1*(self.reduce_patch_size**2)
        d_model3 = d_model2*(self.reduce_patch_size**2)
        dropout_p = 0.01
        self.img_h = img_h
        self.img_w = img_w
        self.conv_scale_up = nn.Sequential(
            nn.Conv2d(3,4,3,padding=1),
            Patchify(patch_size = self.patch_size),
            PositionalEncodingAppend(d_model1, device = mps_device)
        )
        self.res_block_11 = nn.Sequential(
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_12 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_13 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_14 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection1 = MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0])
        self.res_block_21 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = False, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_22 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = False, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = False, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_23 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = False, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = False, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection2 = MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1])
        self.res_block_31 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block3_heads, d_model = d_model3, num_masks = res_block3_masks, reduction = False, mask1 = masks[2][0], mask2 = masks[2][1], roll_matrix = rolls[2]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_32 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block3_heads, d_model = d_model3, num_masks = res_block3_masks, reduction = False, mask1 = masks[2][0], mask2 = masks[2][1], roll_matrix = rolls[2]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block3_heads, d_model = d_model3, num_masks = res_block3_masks, reduction = False, mask1 = masks[2][0], mask2 = masks[2][1], roll_matrix = rolls[2]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_33 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block3_heads, d_model = d_model3, num_masks = res_block3_masks, reduction = False, mask1 = masks[2][0], mask2 = masks[2][1], roll_matrix = rolls[2]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block3_heads, d_model = d_model3, num_masks = res_block3_masks, reduction = False, mask1 = masks[2][0], mask2 = masks[2][1], roll_matrix = rolls[2]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.fcn = nn.Sequential(
            nn.Linear(d_model3, 10)
        )
        self.LogSoftmax = nn.LogSoftmax(dim=1)
    def forward(self, input):
        out0 = self.conv_scale_up(input)
        out1 = self.res_block_11(out0)+out0
        out2 = self.res_block_12(out1)+out1
        out3 = self.res_block_13(out2)+out2
        out4 = self.res_block_14(out3)+out3

        out5 = self.res_block_21(out4)+self.shortcut_projection1(out4)
        out6 = self.res_block_22(out5)+out5
        out7 = self.res_block_23(out6)+out6
        out8 = self.res_block_31(out7)+self.shortcut_projection2(out7)
        out9 = self.res_block_32(out8)+out8
        out10 = self.res_block_33(out9)+out9
        out10 = torch.mean(out10, dim=-2)
        out11 = self.fcn(out10)
        out11 = out11 - torch.amax(out11, 1, keepdim=True)
        out11 = self.LogSoftmax(out11)
        return out11
# model = ConvFormerResNet(img_h = 32, img_w = 32, patch_size=2).to(mps_device)
# conv_input = torch.stack([CIFAR_Conv_Val.__getitem__(random.randint(0, 9999))[0] for i in range(1)]).to(mps_device)
# print(model(conv_input))

In [ ]:
"""
Implementation of Reduced-Size ConvFormers allowing for comparisons with ResNets which have similar amounts of parameters.
"""
patch_size = 2
rolls, masks = gen_rolls_and_masks(img_sizes=[(32,32),(16,16), (8,8)], patch_size = patch_size, filt_type = "filter", mask_fidelity = 3, should_train = True)
class ConvFormerResNetSmall(nn.Module):
    def __init__(self, img_h, img_w, patch_size, masks = masks, rolls = rolls):
        super().__init__()
        self.patch_size = patch_size
        self.reduce_patch_size = 2
        res_block1_heads = 16
        res_block2_heads = res_block1_heads*2
        res_block3_heads = res_block2_heads*2
        res_block1_masks = 16
        res_block2_masks = res_block1_masks*2
        res_block3_masks = res_block2_masks*2
        #Should be square number
        patch_size = 2
        d_model1 = 32
        d_model2 = d_model1*(self.reduce_patch_size**2)
        d_model3 = d_model2*(self.reduce_patch_size**2)
        dropout_p = 0.02
        self.conv_scale_up = nn.Sequential(
            nn.Conv2d(3,4,3,padding=1),
            Patchify(patch_size = self.patch_size),
            PositionalEncodingAppend(d_model1, device = mps_device),
        )
        self.res_block_11 = nn.Sequential(
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_12 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = False, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection1 = MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0])
        self.res_block_21 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block1_heads, d_model = d_model1, num_masks = res_block1_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = False, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection2 = MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1])
        self.res_block_31 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvHead(num_heads = res_block2_heads, d_model = d_model2, num_masks = res_block2_masks, reduction = True, patch_size = self.reduce_patch_size, mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvHead(num_heads = res_block3_heads, d_model = d_model3, num_masks = res_block3_masks, reduction = False, mask1 = masks[2][0], mask2 = masks[2][1], roll_matrix = rolls[2]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.fcn = nn.Sequential(
            nn.Linear(d_model3, 10)
        )
        self.LogSoftmax = nn.LogSoftmax(dim=1)
    def forward(self, input):
        out0 = self.conv_scale_up(input)
        print(out0.shape)
        out1 = self.res_block_11(out0)+out0

        out2 = self.res_block_12(out1)+out1

        out3 = self.res_block_21(out2)+self.shortcut_projection1(out2)

        out4 = self.res_block_31(out3)+self.shortcut_projection2(out3)

        out5 = torch.mean(out4, dim=-2)

        out6 = self.fcn(out5)

        out6 = out6 - torch.amax(out6, 1, keepdim=True)
        out6 = self.LogSoftmax(out6)
        return out6
# model = ConvFormerResNetSmall(img_h = 32, img_w = 32, patch_size=1).to(mps_device)
# conv_input = torch.stack([CIFAR_Conv_Val.__getitem__(random.randint(0, 9999))[0] for i in range(1)]).to(mps_device)
# print(model(conv_input))

In [ ]:
"""
Implementation of ConvFormers that can completely represent convolution. Not covered in paper.
"""
patch_size = 64
rolls, masks = gen_rolls_and_masks_without_repeat(patch_size = patch_size, mask_fidelity = 3, groups = 1)
class ConvFormerResNetFilterSmall(nn.Module):
    def __init__(self, img_h = img_h, img_w = img_w, masks = masks, rolls = rolls):
        super().__init__()
        res_block1_heads = 8
        res_block2_heads = 32
        res_block3_heads = 64
        #Should be square number
        patch_size = 64
        d_model1 = 8
        d_model2 = d_model1*2
        d_model3 = d_model2*2
        dropout_p = 0.02
        self.conv_scale_up = nn.Sequential(
            nn.Conv2d(3,4,3,padding=1),
            Patchify(patch_size = int(math.sqrt(patch_size))),
            PositionalEncodingAppend(d_model1, device = mps_device)
        )
        self.res_block_11 = nn.Sequential(
            MultiScaledConvFilterHead(scale = 1, num_heads = res_block1_heads, d_model = d_model1, seq_len = int(1024/patch_size), reduction = int(1024/patch_size), mask1 = masks[0][0], mask2 = masks[0][1], roll_matrix = rolls[0]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.res_block_12 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvFilterHead(scale = 1, num_heads = res_block1_heads, d_model = d_model1, seq_len = int(1024/patch_size), reduction = int(1024/patch_size), mask1 = masks[1][0], mask2 = masks[1][1], roll_matrix = rolls[1]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvFilterHead(scale = 1, num_heads = res_block1_heads, d_model = d_model1, seq_len = int(1024/patch_size), reduction = int(1024/patch_size), mask1 = masks[2][0], mask2 = masks[2][1], roll_matrix = rolls[2]),
            BatchNormTranspose(d_model1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection1 = MultiScaledConvFilterHead(scale = 2, num_heads = res_block1_heads, d_model = d_model1, seq_len = int(1024/patch_size), reduction = int(256/patch_size), mask1 = masks[3][0], mask2 = masks[3][1], roll_matrix = rolls[3])
        self.res_block_21 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvFilterHead(scale = 2, num_heads = res_block1_heads, d_model = d_model1, seq_len = int(1024/patch_size), reduction = int(256/patch_size), mask1 = masks[4][0], mask2 = masks[4][1], roll_matrix = rolls[4]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvFilterHead(scale = 1, num_heads = res_block2_heads, d_model = d_model2, seq_len = int(256/patch_size), reduction = int(256/patch_size), mask1 = masks[5][0], mask2 = masks[5][1], roll_matrix = rolls[5]),
            BatchNormTranspose(d_model2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.shortcut_projection2 = MultiScaledConvFilterHead(scale = 2, num_heads = res_block2_heads, d_model = d_model2, seq_len = int(256/patch_size), reduction = int(64/patch_size), mask1 = masks[6][0], mask2 = masks[6][1], roll_matrix = rolls[6])
        self.res_block_31 = nn.Sequential(
            nn.ReLU(),
            MultiScaledConvFilterHead(scale = 2, num_heads = res_block2_heads, d_model = d_model2, seq_len = int(256/patch_size), reduction = int(64/patch_size), mask1 = masks[7][0], mask2 = masks[7][1], roll_matrix = rolls[7]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            MultiScaledConvFilterHead(scale = 1, num_heads = res_block3_heads, d_model = d_model3, seq_len = int(64/patch_size), reduction = int(64/patch_size), mask1 = masks[8][0], mask2 = masks[8][1], roll_matrix = rolls[8]),
            BatchNormTranspose(d_model3),
            nn.ReLU(),
            nn.Dropout(dropout_p),
        )
        self.fcn = nn.Sequential(
            nn.Linear(d_model3, 10)
        )
        self.LogSoftmax = nn.LogSoftmax(dim=1)
    def forward(self, input):
        out0 = self.conv_scale_up(input)

        out1 = self.res_block_11(out0)+out0

        out2 = self.res_block_12(out1)+out1

        out3 = self.res_block_21(out2)+self.shortcut_projection1(out2)

        out4 = self.res_block_31(out3)+self.shortcut_projection2(out3)

        out5 = torch.mean(out4, dim=-2)

        out6 = self.fcn(out5)

        out6 = out6 - torch.amax(out6, 1, keepdim=True)
        out6 = self.LogSoftmax(out6)
        return out6
model = ConvFormerResNetFilterSmall().to(mps_device)


In [ ]:
"""
One ResNet block implemented as described in the ResNet paper (He, 2015).
"""
class ResBlock(nn.Module):
    def __init__(self, num_in, num_out):
        super().__init__()
        self.num_in = num_in
        self.num_out = num_out
        self.stride = 1
        self.conv2d_1 = nn.Conv2d(num_in, num_out, 3, stride = self.stride, padding = 1)
        self.conv2d_2 = nn.Conv2d(num_in, num_out, 3, stride = self.stride, padding = 1)
        self.batch_norm = nn.BatchNorm2d(num_out)
        self.ReLU = nn.ReLU()
    def forward(self, input):
        out0 = self.ReLU(input)
        out1 = self.conv2d_1(out0)
        out2 = self.batch_norm(out1)
        out3 = self.ReLU(out2)
        out4 = self.conv2d_2(out3)
        out5 = self.batch_norm(out4)
        return out5

In [ ]:
"""ResNet 20 implementation. Implemented using algorithms described in ResNet paper."""
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.res_block_11 = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding = 1),
            nn.BatchNorm2d(16),
        )
        self.res_block_12 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16)
        )
        self.res_block_13 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16)
        )
        self.res_block_14 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16)
        )
        self.shortcut_projection1 = nn.Conv2d(16, 32, 1, stride = 2)
        self.res_block_21 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding = 1, stride = 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding = 1),
            nn.BatchNorm2d(32),
        )
        self.res_block_22 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding = 1),
            nn.BatchNorm2d(32),
        )
        self.res_block_23 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding = 1),
            nn.BatchNorm2d(32),
        )
        self.shortcut_projection2 = nn.Conv2d(32, 64, 1, stride = 2)
        self.res_block_31 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding = 1, stride = 2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.BatchNorm2d(64)
        )
        self.res_block_32 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.BatchNorm2d(64)
        )
        self.res_block_33 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.BatchNorm2d(64)
        )
        self.pool = torch.nn.AvgPool2d(8, 1)
        self.fcn = nn.Sequential(
            nn.Linear(64, 10)
        )
    def forward(self, input):
        out1 = self.res_block_11(input)
        ReLULayer = nn.ReLU()
        out2 = ReLULayer(self.res_block_12(out1)+out1)
        out3 = ReLULayer(self.res_block_13(out2)+out2)
        out4 = ReLULayer(self.res_block_14(out3)+out3)
        out5 = ReLULayer(self.res_block_21(out4)+self.shortcut_projection1(out4))
        out6 = ReLULayer(self.res_block_22(out5)+out5)
        out7 = ReLULayer(self.res_block_23(out6)+out6)
        out8 = ReLULayer(self.res_block_31(out7)+self.shortcut_projection2(out7))
        out9 = ReLULayer(self.res_block_32(out8)+out8)
        out10 = ReLULayer(self.res_block_33(out9)+out9)
        out10 = self.pool(out10)
        out11 = self.fcn(out10.reshape(out10.shape[0],-1))
        return out11

In [ ]:
"""ResNet with changeable number of layers. Allows for the training of ResNet-20, 56, and 110 by changing the number n. N corresponds with the n in the ResNet paper used to adjust the size of CIFAR-10 networks."""
class ResNetChangeable(nn.Module):
    def __init__(self, n = 9):
        super().__init__()
        self.patch_size = 4
        self.seq_len = 1024/self.patch_size
        self.inp_features = 3*self.patch_size
        self.transform_inp = nn.Sequential(
            Patchify(patch_size = int(math.sqrt(self.patch_size)))
        )
        self.res_block_11 = nn.Sequential(
            nn.Conv2d(self.inp_features, 16, 3, padding = 1),
            nn.BatchNorm2d(16),
        )
        self.res_block_12 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding = 1),
            nn.BatchNorm2d(16)
        )
        self.res_block_1rest = nn.Sequential(
            *[ResBlock(16,16) for i in range(n-1)]
        )
        self.shortcut_projection1 = nn.Conv2d(16, 32, 1, stride = 2)
        self.res_block_21 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding = 1, stride = 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding = 1),
            nn.BatchNorm2d(32),
        )
        self.res_block_2rest = nn.Sequential(
            *[ResBlock(32,32) for i in range(n-1)]
        )
        self.shortcut_projection2 = nn.Conv2d(32, 64, 1, stride = 2)
        self.res_block_31 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding = 1, stride = 2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.BatchNorm2d(64)
        )
        self.res_block_3rest = nn.Sequential(
            *[ResBlock(64,64) for i in range(n-1)]
        )
        self.pool = torch.nn.AvgPool2d(4, 1)
        self.fcn = nn.Sequential(
            nn.Linear(64, 10)
        )
    def forward(self, input):
        out1 = self.transform_inp(input).reshape(-1, int(math.sqrt(self.seq_len)), int(math.sqrt(self.seq_len)), self.inp_features).transpose(-1, 1).transpose(-1,-2)
        out1 = self.res_block_11(out1)
        out2 = self.res_block_12(out1)+out1
        out3 = self.res_block_1rest(out2)+out2
        out4 = self.res_block_21(out3)+self.shortcut_projection1(out3)
        out5 = self.res_block_2rest(out4)+out4
        out6 = self.res_block_31(out5)+self.shortcut_projection2(out5)
        out7 = self.res_block_3rest(out6)+out6
        out8 = self.pool(out7)
        out9 = self.fcn(out8.reshape(out8.shape[0],-1))
        return out9

In [ ]:
"""Find parameter count for reduced size ConvFormers"""
model = ConvFormerResNetSmall(img_h=32, img_w=32, patch_size=2).to(mps_device)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

In [ ]:
"""Find parameter count for ResNets"""
model = ResNetChangeable(n=18).to(mps_device)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

In [ ]:
"""Find parameter count for Vision Transformers"""
model = TransformerResNetSmall(img_h=32, img_w=32).to(mps_device)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

In [ ]:
"""Training loop for the ResNets. Plots the validation accuracy along with the train and validation loss. Uses Learning Rate Plateau, SGD and weight decay"""
import os
from torcheval.metrics import MulticlassAccuracy

test_num = 3
hyperparam_dicts = [{
    'weight_decay': [0.0001, 0.001, 0.01][i],
} for i in range(test_num)]
save_models = False
start_model = 0
if(start_model > 0):
    model.load_state_dict(torch.load(f'ResNet/model{start_model}.pt'))
epochs = 100-start_model
if not os.path.exists("ResNet"):
    os.mkdir("ResNet")
final_accuracies = []
# for hyperparam_dict in hyperparam_dicts:
conv_model = ResNetChangeable(n=18).to(mps_device)
optimizer = torch.optim.SGD(conv_model.parameters(), weight_decay=0.01, lr = 0.01, momentum = 0.9)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 10, mode='min')
loss_fn = nn.CrossEntropyLoss().to(mps_device)
accuracy_fn = MulticlassAccuracy()
epochs = 100
val_accuracy = []
model_loss = []
val_loss = []
grad_mags = []
clip_val = 3
for j in range(epochs):
    plt.show()
    model_sub_loss = torch.zeros((len(CIFAR_Convloader),))
    for i, batch in enumerate(CIFAR_Convloader):
        optimizer.zero_grad()
        inputs = batch[0].to(mps_device)
        labels = batch[1].to(mps_device)
        outputs = conv_model(inputs)
        loss = loss_fn(outputs, labels)
        model_sub_loss[i] = loss.cpu().detach()
        if(i%100==0):
            accuracy_fn.update(outputs, labels)
            print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Convloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
        loss.backward()
        if(i%100 == 0):
            grad_mag = torch.norm(torch.stack([torch.norm(p.grad, 2.0) for p in conv_model.parameters() if p.grad is not None]), 2.0)
            grad_mags.append(grad_mag)
            print(grad_mag)
        torch.nn.utils.clip_grad_norm_(conv_model.parameters(), clip_val)
        optimizer.step()
    model_loss.append(torch.mean(model_sub_loss,dim=-1))
    print("Validation Stage:")

    dropout_modules = [module for module in model.modules() if isinstance(module,torch.nn.Dropout)]
    [module.eval() for module in dropout_modules]
    accuracy_fn.reset()
    k = 0
    val_sub_loss = torch.zeros((len(CIFAR_Valconvloader),))
    for i, batch in enumerate(CIFAR_Valconvloader):
        with torch.no_grad():
            inputs = batch[0]
            labels = batch[1]
            inputs = inputs.to(mps_device)
            labels = labels.to(mps_device)
            outputs = conv_model(inputs)
            loss = loss_fn(outputs, labels)
            val_sub_loss[k] = loss.cpu().detach()
            accuracy_fn.update(outputs, labels)
            if(k%10==0):
                print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Valconvloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
            k+=1
    lr_scheduler.step(torch.mean(val_sub_loss))
    val_loss.append(torch.mean(val_sub_loss))
    print(f"Final Accuracy: {accuracy_fn.compute()}")
    val_accuracy.append(accuracy_fn.compute())
    accuracy_fn.reset()
    plt.plot(model_loss)
    plt.plot(val_loss)
    plt.show()
    plt.plot(val_accuracy)
    plt.show()
    plt.imshow(inputs[0].permute(1,2,0).cpu().detach())
    plt.show()
    [module.train() for module in dropout_modules]
    torch.save(model.state_dict(),f"ResNet/model{j}.pt")




In [ ]:
"""Save training state of ConvFormers"""
torch.save(model.state_dict(),f"/ConvFormerResNet1/model{j}_final.pt")
torch.save(optimizer.state_dict(),f"/ConvFormerResNet1/model{j}_optim.pt")
torch.save(lr_scheduler.state_dict(),f"/ConvFormerResNet1/model{j}_scheduler.pt")
"""Save ResNet metrics"""
with open("ResNet/grad_mags.txt", 'w+') as writer:
    for grad_mag in grad_mags:
        writer.write(f"{grad_mag},")
with open("ResNet/val_loss.txt", 'w+') as writer:
    for loss in val_loss:
        writer.write(f"{loss},")
with open("ResNet/train_loss.txt", 'w+') as writer:
    for loss in model_loss:
        writer.write(f"{loss},")
with open("ResNet/val_accuracy.txt", 'w+') as writer:
    for accuracy in val_accuracy:
        writer.write(f"{accuracy},")
plt.plot(model_loss)
plt.plot(val_loss)
plt.savefig(f"ResNet/TrainVsValidationLoss.png")
plt.show()
plt.plot(val_accuracy)
plt.savefig(f"ResNet/ValidationAccuracy.png")

In [ ]:
"""Zip ResNet models"""
!zip -r /ResNetModel20PatchedPt.zip /ResNet/*_*.pt

In [ ]:
"""Zip ResNet metrics"""
!zip -r /ResNetModel20PatchedTxt.zip /ResNet/*.txt

In [ ]:
"""Training loop for ConvFormers. Plots the validation accuracy along with the train and validation loss. Uses Learning Rate Plateau, SGD and weight decay. Also plots one learnable mask to show the effect of GradientBiasing"""
import os
from torcheval.metrics import MulticlassAccuracy
model = ConvFormerResNetSmall(img_h = 32, img_w = 32, patch_size=2).to(mps_device)
optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.003, lr = 0.01, momentum = 0.9)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience = 10)
start_model = 0
if not os.path.exists("ConvFormerResNet"):
    os.mkdir("ConvFormerResNet")
if(start_model > 0):
    model.load_state_dict(torch.load(f'ConvFormerResNet/model{start_model-1}.pt'))
    optimizer.load_state_dict(torch.load(f'ConvFormerResNet/model{start_model-1}_optim.pt'))
    lr_scheduler.load_state_dict(torch.load(f'ConvFormerResNet/model{start_model-1}_scheduler.pt'))
loss_fn = nn.NLLLoss().to(mps_device)
accuracy_fn = MulticlassAccuracy()
epochs = 100-start_model
val_accuracy = []
model_loss = []
val_loss = []
grad_mags = []
clip_val = 3
start_batch = 0
prev_matrix = torch.gather(torch.matmul(model.res_block_11[0].mask1.transpose(-1,-2),model.res_block_11[0].mask2).cpu().detach()[0], -1, model.res_block_11[0].roll_back.cpu().detach())
plt.matshow(prev_matrix)
plt.show()
for j in range(epochs):
    model_sub_loss = torch.zeros((len(CIFAR_Convloader),))
    model_sub_loss.requires_grad = False
    for i, batch in enumerate(CIFAR_Convloader):
        if j == 0 and i < start_batch:
            continue
        optimizer.zero_grad(set_to_none=True)
        inputs = batch[0]
        labels = batch[1]
        inputs = inputs.to(mps_device)
        labels = labels.to(mps_device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        model_sub_loss[i] = loss.cpu().detach()
        if(i%100==0):
            accuracy_fn.update(outputs, labels)
            print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Convloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
        loss.backward()
        if(i%100 == 0):
            grad_mag = torch.norm(torch.stack([torch.norm(p.grad, 2.0) for p in model.parameters() if p.grad is not None]), 2.0)
            grad_mags.append(grad_mag)
            print(grad_mag)
            torch.cuda.empty_cache()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_val)
        optimizer.step()
    model_loss.append(torch.mean(model_sub_loss,dim=-1))
    print("Validation Stage:")
    dropout_modules = [module for module in model.modules() if isinstance(module,torch.nn.Dropout)]
    [module.eval() for module in dropout_modules]
    accuracy_fn.reset()
    k = 0
    val_sub_loss = torch.zeros((len(CIFAR_Valconvloader),))
    val_sub_loss.requires_grad = False
    for i, batch in enumerate(CIFAR_Valconvloader):
        with torch.no_grad():
            inputs = batch[0]
            labels = batch[1]
            inputs = inputs.to(mps_device)
            labels = labels.to(mps_device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            val_sub_loss[k] = loss.cpu().detach()
            accuracy_fn.update(outputs, labels)
            if(k%10==0):
                print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Valconvloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
            k+=1
    print(f"Final Accuracy: {accuracy_fn.compute()}")
    lr_scheduler.step(torch.mean(val_sub_loss))
    val_loss.append(torch.mean(val_sub_loss))
    val_accuracy.append(accuracy_fn.compute())
    accuracy_fn.reset()
    plt.plot(model_loss)
    plt.plot(val_loss)
    plt.show()
    plt.plot(val_accuracy)
    plt.show()
    plt.imshow(inputs[0].permute(1,2,0).cpu().detach())
    plt.show()
    plt.matshow(torch.gather(torch.matmul(model.res_block_11[0].mask1.transpose(-1,-2),model.res_block_11[0].mask2).cpu().detach()[0], -1, model.res_block_11[0].roll_back.cpu().detach()))
    plt.show()
    print(f"lr: {lr_scheduler._last_lr }")
    print(f"mask_shape = {model.res_block_11[0].mask1.shape}")
    [module.train() for module in dropout_modules]
    torch.save(model.state_dict(),f"ConvFormerResNet/model{j+start_model}.pt")


In [ ]:
"""Save training state of ConvFormers"""
torch.save(model.state_dict(),f"ConvFormerResNet/model{j+start_model-1}_final.pt")
torch.save(optimizer.state_dict(),f"ConvFormerResNet/model{j+start_model-1}_optim.pt")
torch.save(lr_scheduler.state_dict(),f"ConvFormerResNet/model{j+start_model-1}_scheduler.pt")

In [ ]:
"""Save ConvFormer metrics"""
if not os.path.exists("ConvFormerResNet"):
    os.mkdir("ConvFormerResNet")
with open("ConvFormerResNet/grad_mags.txt", 'w+') as writer:
    for grad_mag in grad_mags:
        writer.write(f"{grad_mag},")
with open("ConvFormerResNet/val_loss.txt", 'w+') as writer:
    for loss in val_loss:
        writer.write(f"{loss},")
with open("ConvFormerResNet/train_loss.txt", 'w+') as writer:
    for loss in model_loss:
        writer.write(f"{loss},")
with open("ConvFormerResNet/val_accuracy.txt", 'w+') as writer:
    for accuracy in val_accuracy:
        writer.write(f"{accuracy},")

In [ ]:
"""Training loop for ConvFormers. Plots the validation accuracy along with the train and validation loss. Uses Learning Rate Plateau, SGD and weight decay. Also plots one learnable mask to show the effect of GradientBiasing"""
import os
from torcheval.metrics import MulticlassAccuracy
model = TransformerResNetSmall(img_h = 32, img_w = 32, patch_size=2).to(mps_device)
optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.001, lr = 0.1, momentum = 0.9)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience = 10)
start_model = 0
if not os.path.exists("TransformerResNet"):
    os.mkdir("TransformerResNet")
if(start_model > 0):
    model.load_state_dict(torch.load(f'TransformerResNet/model{start_model-1}.pt'))
    optimizer.load_state_dict(torch.load(f'TransformerResNet/model{start_model-1}_optim.pt'))
    lr_scheduler.load_state_dict(torch.load(f'TransformerResNet/model{start_model-1}_scheduler.pt'))
loss_fn = nn.NLLLoss().to(mps_device)
accuracy_fn = MulticlassAccuracy()
epochs = 100-start_model
val_accuracy = []
model_loss = []
val_loss = []
grad_mags = []
clip_val = 3
start_batch = 0
for j in range(epochs):
    model_sub_loss = torch.zeros((len(CIFAR_Convloader),))
    model_sub_loss.requires_grad = False
    for i, batch in enumerate(CIFAR_Convloader):
        if j == 0 and i < start_batch:
            continue
        optimizer.zero_grad(set_to_none=True)
        inputs = batch[0]
        labels = batch[1]
        inputs = inputs.to(mps_device)
        labels = labels.to(mps_device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        model_sub_loss[i] = loss.cpu().detach()
        if(i%100==0):
            accuracy_fn.update(outputs, labels)
            print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Convloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
        loss.backward()
        if(i%100 == 0):
            grad_mag = torch.norm(torch.stack([torch.norm(p.grad, 2.0) for p in model.parameters() if p.grad is not None]), 2.0)
            grad_mags.append(grad_mag)
            print(grad_mag)
            torch.cuda.empty_cache()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_val)
        optimizer.step()
    model_loss.append(torch.mean(model_sub_loss,dim=-1))
    print("Validation Stage:")
    dropout_modules = [module for module in model.modules() if isinstance(module,torch.nn.Dropout)]
    [module.eval() for module in dropout_modules]
    accuracy_fn.reset()
    k = 0
    val_sub_loss = torch.zeros((len(CIFAR_Valconvloader),))
    val_sub_loss.requires_grad = False
    for i, batch in enumerate(CIFAR_Valconvloader):
        with torch.no_grad():
            inputs = batch[0]
            labels = batch[1]
            inputs = inputs.to(mps_device)
            labels = labels.to(mps_device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            val_sub_loss[k] = loss.cpu().detach()
            accuracy_fn.update(outputs, labels)
            if(k%10==0):
                print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Valconvloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
            k+=1
    print(f"Final Accuracy: {accuracy_fn.compute()}")
    lr_scheduler.step(torch.mean(val_sub_loss))
    val_loss.append(torch.mean(val_sub_loss))
    val_accuracy.append(accuracy_fn.compute())
    accuracy_fn.reset()
    plt.plot(model_loss)
    plt.plot(val_loss)
    plt.show()
    plt.plot(val_accuracy)
    plt.show()
    plt.imshow(inputs[0].permute(1,2,0).cpu().detach())
    plt.show()
    print(f"lr: {lr_scheduler._last_lr }")
    [module.train() for module in dropout_modules]
    torch.save(model.state_dict(),f"TransformerResNet/model{j+start_model}.pt")


In [ ]:
"""Save training state of ConvFormers"""
torch.save(model.state_dict(),f"TransformerResNet/model{j+start_model-1}_final.pt")
torch.save(optimizer.state_dict(),f"TransformerResNet/model{j+start_model-1}_optim.pt")
torch.save(lr_scheduler.state_dict(),f"TransformerResNet/model{j+start_model-1}_scheduler.pt")

In [ ]:
"""Save ConvFormer metrics"""
if not os.path.exists("TransformerResNet"):
    os.mkdir("TransformerResNet")
with open("TransformerResNet/grad_mags.txt", 'w+') as writer:
    for grad_mag in grad_mags:
        writer.write(f"{grad_mag},")
with open("TransformerResNet/val_loss.txt", 'w+') as writer:
    for loss in val_loss:
        writer.write(f"{loss},")
with open("TransformerResNet/train_loss.txt", 'w+') as writer:
    for loss in model_loss:
        writer.write(f"{loss},")
with open("TransformerResNet/val_accuracy.txt", 'w+') as writer:
    for accuracy in val_accuracy:
        writer.write(f"{accuracy},")

In [ ]:
"""Training loop for Transformers and Vision Transformers. Plots the validation accuracy along with the train and validation loss. Uses Learning Rate Plateau, SGD and weight decay."""
import os
from torcheval.metrics import MulticlassAccuracy
model = VisionTransformer(img_h = 32, img_w = 32).to(mps_device)
optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.01, lr = 0.01, momentum = 0.9)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience = 10)
start_model = 0
if(start_model > 0):
    model.load_state_dict(torch.load(f'VisionTransformer/model{start_model-1}.pt'))
    optimizer.load_state_dict(torch.load(f'VisionTransformer/model{start_model-1}_optim.pt'))
    lr_scheduler.load_state_dict(torch.load(f'VisionTransformer/model{start_model-1}_scheduler.pt'))
loss_fn = nn.CrossEntropyLoss().to(mps_device)
accuracy_fn = MulticlassAccuracy()
epochs = 100-start_model
val_accuracy = []
model_loss = []
val_loss = []
grad_mags = []
if not os.path.exists("VisionTransformer"):
    os.mkdir("VisionTransformer")
for j in range(epochs):
    model_sub_loss = torch.zeros((len(CIFAR_Convloader),))
    model_sub_loss.requires_grad = False
    for i, batch in enumerate(CIFAR_Convloader):
        optimizer.zero_grad()
        inputs = batch[0]
        labels = batch[1]
        inputs = inputs.to(mps_device)
        labels = labels.to(mps_device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        model_sub_loss[i] = loss.cpu().detach()
        if(i%100==0):
            accuracy_fn.update(outputs, labels)
            print(labels)
            print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Convloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
        loss.backward()
        if(i%100 == 0):
            grad_mag = torch.norm(torch.stack([torch.norm(p.grad, 2.0) for p in model.parameters() if p.grad is not None]), 2.0)
            grad_mags.append(grad_mag)
            print(grad_mag)
            torch.cuda.empty_cache()
        optimizer.step()
    model_loss.append(torch.mean(model_sub_loss,dim=-1))
    print("Validation Stage:")
    dropout_modules = [module for module in model.modules() if isinstance(module,torch.nn.Dropout)]
    [module.eval() for module in dropout_modules]
    accuracy_fn.reset()
    k = 0
    val_sub_loss = torch.zeros((len(CIFAR_Valconvloader),))
    val_sub_loss.requires_grad = False
    for i, batch in enumerate(CIFAR_Valconvloader):
        with torch.no_grad():
            inputs = batch[0]
            labels = batch[1]
            inputs = inputs.to(mps_device)
            labels = labels.to(mps_device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            val_sub_loss[k] = loss.cpu().detach()
            accuracy_fn.update(outputs, labels)
            if(k%10==0):
                print(labels)
                print(f"Loss: {loss}, Batch Num: {i}/{len(CIFAR_Valconvloader)}, Accuracy:{accuracy_fn.compute()}, Epoch: {j+start_model}")
            k+=1
    print(f"Final Accuracy: {accuracy_fn.compute()}")
    lr_scheduler.step(torch.mean(val_sub_loss))
    val_loss.append(torch.mean(val_sub_loss))
    val_accuracy.append(accuracy_fn.compute())
    accuracy_fn.reset()
    plt.plot(model_loss)
    plt.plot(val_loss)
    plt.show()
    plt.plot(val_accuracy)
    plt.show()
    plt.imshow(inputs[0].permute(1,2,0).cpu().detach())
    plt.show()
    [module.train() for module in dropout_modules]
    torch.save(model.state_dict(),f"VisionTransformer/model{j+start_model}.pt")


In [ ]:
"""Save VisionTransformer training state"""
torch.save(model.state_dict(),f"VisionTransformer/model{j+start_model-1}_final.pt")
torch.save(optimizer.state_dict(),f"VisionTransformer/model{j+start_model-1}_optim.pt")
torch.save(lr_scheduler.state_dict(),f"VisionTransformer/model{j+start_model-1}_scheduler.pt")

In [ ]:
"""Save VisionTransformer metrics"""
with open("VisionTransformer/grad_mags.txt", 'w+') as writer:
    for grad_mag in grad_mags:
        writer.write(f"{grad_mag},")
with open("VisionTransformer/val_loss.txt", 'w+') as writer:
    for loss in val_loss:
        writer.write(f"{loss},")
with open("VisionTransformer/train_loss.txt", 'w+') as writer:
    for loss in model_loss:
        writer.write(f"{loss},")
with open("VisionTransformer/accuracy.txt", 'w+') as writer:
    for accuracy in val_accuracy:
        writer.write(f"{val_accuracy},")

In [ ]:
"""Zip Vision Transformer metrics"""
!zip -r /VisionTransformerTxt.zip /VisionTransformer/*.txt

In [ ]:
"""Zip Vision Transformer model"""
!zip -r /VisionTransformerModel.zip /VisionTransformer/*_*.pt